<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%2010/cis530_nmt_skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/Drive', force_remount = True)
# When you run this block, you will need to click open a link to get some 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [0]:
import os
# TODO
os.chdir('Drive/My Drive/CIS-530/Homework 10/data')

In [0]:
# Preprocess data to OpenNMT format
for fname in ['train','dev','test']:
  # TODO: create 6 files, English|Japanese data for train|dev|test, one sentence per line.
  with open('split/' + fname, encoding='utf-8') as fr, open('split/en-' + fname + '.txt', 'w', encoding='utf-8') as fwe, open('split/ja-' + fname + '.txt', 'w', encoding='utf-8') as fwj:
    for line in fr:
      split_line = line.split('\t')
      english = split_line[0]
      japanese = split_line[1]
      fwe.write(english + '\n')
      fwj.write(japanese)

In [0]:
# Install BPE (https://github.com/rsennrich/subword-nmt)
# TODO
!pip install subword-nmt

In [0]:
# Apply BPE, following instructions from the Github repo
# this takes around 15 minutes

# TODO
# codes_file is where the BPE information is stored. Name it as you like. 

# Learn and apply BPE for English data

!subword-nmt learn-bpe -s 10000 < 'split/en-train.txt' > 'bpe/en-bpe' # learn from train
!subword-nmt apply-bpe -c 'bpe/en-bpe' < 'split/en-train.txt' > 'bpe/en-train-bpe-out.txt' # apply to train
!subword-nmt apply-bpe -c 'bpe/en-bpe' < 'split/en-dev.txt' > 'bpe/en-dev-bpe-out.txt' # apply to dev
!subword-nmt apply-bpe -c 'bpe/en-bpe' < 'split/en-test.txt' > 'bpe/en-test-bpe-out.txt' # apply to test

In [0]:
# Learn and apply BPE for Japanese data

!subword-nmt learn-bpe -s 10000 < 'split/ja-train.txt' > 'bpe/ja-bpe' # learn from train
!subword-nmt apply-bpe -c 'bpe/ja-bpe' < 'split/ja-train.txt' > 'bpe/ja-train-bpe-out.txt' # apply to train
!subword-nmt apply-bpe -c 'bpe/ja-bpe' < 'split/ja-dev.txt' > 'bpe/ja-dev-bpe-out.txt' # apply to dev
!subword-nmt apply-bpe -c 'bpe/ja-bpe' < 'split/ja-test.txt' > 'bpe/ja-test-bpe-out.txt' # apply to test

In [0]:
# Install OpenNMT (https://github.com/OpenNMT/OpenNMT-py)
# TODO
!pip install OpenNMT-py

In [0]:
# Run OpenNMT, following instructions from the Github repo
# this takes 1-2 hours

# TODO
# Name the prefixes for data and checkpoints as you like
!onmt_preprocess -train_src 'bpe/ja-train-bpe-out.txt' -train_tgt 'bpe/en-train-bpe-out.txt' -valid_src 'bpe/ja-dev-bpe-out.txt' -valid_tgt 'bpe/en-dev-bpe-out.txt' -save_data 'bpe/demo' -overwrite

In [0]:
!onmt_train -gpu_ranks 0 -data bpe/Demo/demo -save_model demo-model

In [0]:
!onmt_translate -model demo-model_step_100000.pt -src 'bpe/ja-test-bpe-out.txt' -output 'bpe/pred-bpe.txt' -replace_unk -verbose

In [0]:
# reverse BPE
!sed -r 's/(@@ )|(@@ ?$)//g' 'bpe/pred-bpe.txt' > 'bpe/pred.txt'

In [0]:
# Evaluate translations using BLEU
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/generic/multi-bleu.perl
# TODO
!perl multi-bleu.perl 'split/en-test.txt' < 'bpe/pred.txt'

In [0]:
import subprocess

with open('split/en-test.txt', encoding = 'utf-8') as ref_file:
  for idx, line in enumerate(ref_file):
    if idx < 20:
      line_file = 'ref_line_%d.txt' % idx
      with open('bpe/lines/ref_lines/' + line_file, 'w', encoding = 'utf-8') as lf:
        lf.write(line)

with open('bpe/pred.txt', encoding = 'utf-8') as pred_file:
  for idx, line in enumerate(pred_file):
    if idx < 20:
      line_file = 'pred_line_%d.txt' % idx
      with open('bpe/lines/pred_lines/' + line_file, 'w', encoding = 'utf-8') as lf:
        lf.write(line)

for idx in range(20):
  ref_file = 'bpe/lines/ref_lines/ref_line_%d.txt' % idx
  pred_file = 'bpe/lines/pred_lines/pred_line_%d.txt' % idx
  with open(ref_file, encoding = 'utf-8') as ref:
    for line in ref:
      print(idx, line)
  with open(pred_file, encoding = 'utf-8') as pred:
    for line in pred:
      print(idx, line)
  cp = subprocess.run("perl multi-bleu.perl '%s' < '%s'" % (ref_file, pred_file), universal_newlines = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE, shell = True)
  print(cp.stdout)

In [0]:
bleu_indiv = [0, 100, 0, 43.47, 0, 0, 0, 0, 34.57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
bleu = 5.84
# Adequacy: how much of the meaning in the gold standard is present in the translation?
adequacy = [0, 3, 3, 2, 1, 2, 1, 2, 3, 3, 0, 0, 2, 1, 2, 0, 0, 2, 3, 0]
# Fluency: to what extent is the translation error free, grammatically as well as spelling wise?
fluency = [3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 0, 3, 3, 1, 3, 1, 3, 3, 3, 1]

In [0]:
!pip install rouge
from rouge import FilesRouge

files_rouge = FilesRouge()

hyp_path = 'bpe/pred.txt' 
ref_path = 'split/en-test.txt'
scores_indiv = files_rouge.get_scores(hyp_path, ref_path)
scores_avg = files_rouge.get_scores(hyp_path, ref_path, avg = True)

In [0]:
for idx in range(20):
  print(scores_indiv[:20][idx]['rouge-l']['f'])

In [0]:
rouge_indiv = [0, 1, 1, 0.77, 0.50, 0, 0.14, 0.50, 0.57, 0.40, 0, 0.27, 0, 0.15, 0, 0, 0.12, 0.29, 0.29, 0.40]
rouge = 0.27 # from scores_avg

In [0]:
meteor_indiv = [0, 1, 1, 0.64, 0.15, 0, 0.03, 0.25, 0.55, 0.11, 0, 0.22, 0, 0.04, 0, 0, 0.04, 0.06, 0.19, 0.09]
meteor = 0.15

In [0]:
from scipy import stats

print(stats.spearmanr(bleu_indiv, adequacy))
print(stats.spearmanr(bleu_indiv, fluency))

print('\n')

print(stats.spearmanr(rouge_indiv, adequacy))
print(stats.spearmanr(rouge_indiv, fluency))

print('\n')

print(stats.spearmanr(meteor_indiv, adequacy))
print(stats.spearmanr(meteor_indiv, fluency))

SpearmanrResult(correlation=0.42656785213142046, pvalue=0.06070997236638958)
SpearmanrResult(correlation=0.23999539213270613, pvalue=0.3081184349083537)


SpearmanrResult(correlation=0.5545437093737067, pvalue=0.011164009612464271)
SpearmanrResult(correlation=0.3230576734035238, pvalue=0.16473587826692543)


SpearmanrResult(correlation=0.5343389780851688, pvalue=0.015222348878875979)
SpearmanrResult(correlation=0.3994745602521892, pvalue=0.08098955833369174)


In [0]:
# Run NMT system on jp_test_sentences.txt 

# BPE for jpe_test_sentences.txt
!subword-nmt apply-bpe -c 'bpe/ja_bpe' < 'jp_test_sentences.txt' > 'jp-test-sentences-bpe-out.txt' # apply to test
!onmt_translate -model 'bpe/demo-model_step_100000.pt' -src 'jp-test-sentences-bpe-out.txt' -output 'translations-bpe.txt' -replace_unk -verbose
!sed -r 's/(@@ )|(@@ ?$)//g' 'translations-bpe.txt' > 'translations.txt'